# TripleGAN - MNIST + Keras

## 0. Imports

In [1]:
# Maths
import numpy                                            as np

# Matplotlib
import matplotlib                                       as mp
import matplotlib.pyplot                                as pt

# Machine / Deep Learning
import tensorflow                                       as tf
import tensorflow_addons                                as tfa
import keras                                            as ks
from keras              import models                   as mdls
from keras              import layers                   as lyrs
from keras.datasets     import mnist                    as mn
from keras.utils        import to_categorical           as tc

# Versions
print( f"Numpy .... : {np.__version__}" )
print( f"Matplotlib : {mp.__version__}" )
print( f"Tensorflow : {tf.__version__}" )
print( f"TF Addons  : {tfa.__version__}" )
print( f"Keras .... : {ks.__version__}" )

Numpy .... : 1.19.2
Matplotlib : 3.3.2
Tensorflow : 2.4.0
TF Addons  : 0.12.0
Keras .... : 2.4.3


## 1. Set Up Model

In [2]:
image_shape = ( 28, 28, 1 )
classes = 10
latent_dim = 100
optimizer = tf.optimizers.Adam( 0.0002, 0.5 )
losses = ['binary_crossentropy','sparse_categorical_crossentropy']

In [3]:
# Discriminator Constants

noise_dis = 0.2
alpha_dis = 0.02


# Discriminator Functions

def lyrs_Noise( ):
    return ks.layers.GaussianNoise( stddev = noise_dis )

def lyrs_Dense( units, activation = None ):
    return ks.layers.Dense( units = units, activation = activation )

def lyrs_WeightNorm( x ):
    return tfa.layers.WeightNormalization( x )

def lyrs_WeightNormDense( units, activation = None ):
    return lyrs_WeightNorm( lyrs_Dense( units = units, activation = activation ) )

def lyrs_lReLU( ):
    return ks.layers.LeakyReLU( alpha = alpha_dis )

### 1.0 Set Up Discriminator

In [14]:
# Set up Discriminator

dr = mdls.Sequential( name = "Discriminator" )

dr.add( lyrs.InputLayer( input_shape = ( 28, 28, ) ) )
dr.add( lyrs.InputLayer( input_shape = ( 10, ) ) )
dr.add( lyrs_Noise() )
dr.add( lyrs_WeightNormDense( 1000 ) )
dr.add( lyrs_lReLU() )
dr.add( lyrs_Noise() )
dr.add( lyrs_WeightNormDense( 500 ) )
dr.add( lyrs_lReLU() )
dr.add( lyrs_Noise() )
dr.add( lyrs_WeightNormDense( 250 ) )
dr.add( lyrs_lReLU() )
dr.add( lyrs_Noise() )
dr.add( lyrs_WeightNormDense( 250 ) )
dr.add( lyrs_lReLU() )
dr.add( lyrs_Noise() )
dr.add( lyrs_WeightNormDense( 250 ) )
dr.add( lyrs_lReLU() )
dr.add( lyrs_Noise() )
dr.add( lyrs_WeightNormDense( 1, activation = "sigmoid" ) )

### 1.1 Set Up Generator

In [15]:
# Set up Generator

gr = mdls.Sequential( name = "Generator" )

gr.add( lyrs.InputLayer( input_shape = ( latent_dim, 28, 28 ) ) )
gr.add( lyrs.Dense( 500, activation = "softplus" ) )
gr.add( lyrs.BatchNormalization( ) )
gr.add( lyrs.Dense( 500, activation = "softplus" ) )
gr.add( lyrs.BatchNormalization( ) )
gr.add( lyrs.Dense( 784, activation = "sigmoid" ) )
gr.add( lyrs.BatchNormalization( ) )

### 1.2 Set Up Classifier

In [12]:
# Set up Classifier

cr = mdls.Sequential( name = "Classifier" )

cr.add( lyrs.Input( shape = ( latent_dim, 28, 28 ) ) )
cr.add( lyrs.Conv2D( 32, kernel_size = ( 5, 5 ), activation = "relu", padding = "same" ) )
cr.add( lyrs.MaxPooling2D( pool_size = ( 2, 2 ) ) )
cr.add( lyrs.Dropout( 0.5 ) )
cr.add( lyrs.Conv2D( 64, kernel_size = ( 3, 3 ), activation = "relu", padding = "same" ) )
cr.add( lyrs.Conv2D( 64, kernel_size = ( 3, 3 ), activation = "relu", padding = "same" ) )
cr.add( lyrs.MaxPooling2D( pool_size = ( 2, 2 ) ) )
cr.add( lyrs.Dropout( 0.5 ) )
cr.add( lyrs.Conv2D( 128, kernel_size = ( 3, 3 ), activation = "relu", padding = "same" ) )
cr.add( lyrs.Conv2D( 128, kernel_size = ( 3, 3 ), activation = "relu", padding = "same" ) )
cr.add( lyrs.GlobalMaxPooling2D() )
cr.add( lyrs.Dense( 10, activation = "softmax" ) )

In [13]:
# Summarize

dr.summary()
gr.summary()
cr.summary()

Model: "Discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         multiple                  0         
_________________________________________________________________
gaussian_noise (GaussianNois (None, None, 784)         0         
_________________________________________________________________
weight_normalization (Weight (None, None, 1000)        1571001   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, None, 1000)        0         
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, None, 1000)        0         
_________________________________________________________________
weight_normalization_1 (Weig (None, None, 500)         1001501   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, None, 500)       

In [ ]:
class TripleGAN( keras.Model ):
    def __init__( self, discriminator, generator, classifier, dimension ):
        super( MonoGAN, self ).__init__()
        self.discriminator = discriminator
        self.classifier = classifier
        self.generator = generator
        self.dimension = dimension

    def compile( self, dr_opt, gr_opt, loss_function ):
        super( MonoGAN, self ).compile()
        self.dr_opt = dr_opt
        self.gr_opt = gr_opt
        self.loss_function = loss_function

    def train_step( self, real_images ):

        if isinstance( real_images, tuple ):
            real_images = real_images[0]

        # Sample random points in the latent space
        batch_size = tf.shape( real_images )[0]
        random_latent_vectors = tf.random.normal( shape = ( batch_size, self.dimension ) )

        # Decode them to fake images
        generated_images = self.generator( random_latent_vectors )

        # Combine them with real images
        combined_images = tf.concat(
            [generated_images, real_images],
            axis = 0
        )

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones( ( batch_size, 1 ) ), tf.zeros( ( batch_size, 1 ) )],
            axis = 0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform( tf.shape( labels ) )

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator( combined_images )
            dr_loss = self.loss_function( labels, predictions )
        
        grads = tape.gradient( dr_loss, self.discriminator.trainable_weights )
        self.dr_opt.apply_gradients(
            zip( grads, self.discriminator.trainable_weights )
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal( shape = ( batch_size, self.dimension ) )

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros( ( batch_size, 1 ) )

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            gr_loss = self.loss_function(misleading_labels, predictions)

        grads = tape.gradient( gr_loss, self.generator.trainable_weights )
        self.gr_opt.apply_gradients( zip( grads, self.generator.trainable_weights ) )

        return {
            "dr_loss": dr_loss,
            "gr_loss": gr_loss
        }


In [ ]:
class TripleGAN_Viewer( ks.callbacks.Callback ):
    def __init__( self, image_number = 100, dimension = 128 ):
        self.image_number = image_number
        self.dimension  = dimension

    def on_epoch_end( self, epoch, logs = None ):
        random_latent_vectors = tf.random.normal( shape = ( self.image_number, self.dimension ) )
        generated_images = self.model.generator( random_latent_vectors )
        generated_images *= 255
        generated_images.numpy()
        for i in range( self.image_number ):
            img = ks.preprocessing.image.array_to_img( generated_images[i] )
            img.save( "generated_img_{i}_{epoch}.png".format( i = i, epoch = epoch ) )


## 2. Setup MNIST Dataset 

In [38]:
( tr_images, tr_labels ), ( ts_images, ts_labels ) = mn.load_data()

batch_size = 100

#all_digits = np.concatenate( [tr_images, ts_images] )
all_digits = tr_images[:100]
all_digits = ( all_digits.astype( 'float32' ) - 127.5 ) / 127.5
all_digits = np.reshape( all_digits, ( -1, 28, 28, 1 ) )

dataset = tf.data.Dataset.from_tensor_slices( all_digits )
dataset = dataset.shuffle( buffer_size=1024 ).batch( batch_size ).prefetch( 32 )

## 3. Create GAN Model

In [33]:
epochs = 30

gan = MonoGAN(
    discriminator   = dr,
    generator       = gr,
    dimension       = dimension
)

## 4.  Compile GAN Model

In [34]:
gan.compile(
    dr_opt          = ks.optimizers.Adam( learning_rate = 0.0003 ),
    gr_opt          = ks.optimizers.Adam( learning_rate = 0.0003 ),
    loss_function   = ks.losses.BinaryCrossentropy( from_logits = True )
)


## 4. Train Model

In [20]:
gan.fit(
    dataset,
    epochs = epochs,
    callbacks = [ MonoGAN_Monitor( image_number = 3, dimension = dimension ) ]
)

Epoch 1/5
938/938 [==============================] - 66s 69ms/step - loss: 0.8713 - accuracy: 0.8633
Epoch 2/5
938/938 [==============================] - 60s 64ms/step - loss: 0.0243 - accuracy: 0.9948
Epoch 3/5
938/938 [==============================] - 41s 43ms/step - loss: 0.0150 - accuracy: 0.9967
Epoch 4/5
938/938 [==============================] - 46s 49ms/step - loss: 0.0168 - accuracy: 0.9961
Epoch 5/5
938/938 [==============================] - 44s 47ms/step - loss: 0.0163 - accuracy: 0.9964


## 5. Evaluate Model

In [22]:
for i in range( 3 ):
    pt.figure( figsize = ( 30, 30 ) )
    im = pt.imread( f"./generated_img_{i}_29.png" )
    pt.imshow( im )

313/313 [==============================] - 2s 8ms/step - loss: 0.0508 - accuracy: 0.9910
0.9909999966621399
